# step2022 -week4-

# PageRankの実装

### 1. 考え方

あるページから次のページへの遷移は、ページ内のリンクをクリックすることで行われることが多いが、ある一定の確率(今回は15%)で、ランダムなアクセスが行われると言われている。そこで、以下のように実装を行う。


1. 全てのノードに適当にスコアを与える。

2. 各頂点のスコアの85%を、隣接するノードに等分して分配する。また、ランダムなアクセスとして、15%のスコアを全てのノードに等分して分配する。<br>
(隣接するノードがない場合、任意のノードに等確率で遷移するものとして、スコアを全てのノードに等分して分配する。)
3. 2を各ノードのスコアが変化しなくなるまで繰り返す。

まずは小さいファイル（links_small.txt, pages_small.txt）で実装

In [1]:
#各ファイルの読み込み
id_to_title = {}
id_to_nextIDs = {}

pages_data = "data/pages_small.txt"
links_data = "data/links_small.txt"

with open(pages_data) as f:
    for data in f.read().splitlines():
        page = data.split("\t")
        id_to_title[page[0]] = page[1]

with open(links_data) as f:
    for data in f.read().splitlines():
        link = data.split("\t")
        if link[0] in id_to_nextIDs:
            id_to_nextIDs[link[0]].add(link[1])
        else:
            id_to_nextIDs[link[0]] = {link[1]}

            
#行列の計算で都合が良くなるよう、ID=>indexの辞書を作成
id_to_index = {}

for i, Id in enumerate(id_to_title.keys()):
    id_to_index[Id] = i

### 2. スコアを分け与える割合の行列を作る

1. ノードの個数をNとしたときに、行列Aを以下のように定める。

    $A=\begin{pmatrix}
    a_{11} & \cdots & a_{1j} & \cdots & a_{1N}\\
    \vdots & \ddots & & & \vdots \\
    a_{i1} & & a_{ij} & & a_{N} \\
    \vdots & & & \ddots & \vdots \\
    a_{N1} & \cdots & a_{Nj} & \cdots & a_{NN}
    \end{pmatrix}$

    ($a_{ij}$はノードiからノードjへ繋がっていたら1、そうでなければ0)
<br>
2. Aの各行の値を合計値で割り、各行の値の合計を1にする。
<br>
3. 行の合計値が0(隣接するノードがない)の場合は1/Nで埋める。
<br>
4. 1,2,3で施した行列を$B$、全ての成分が1/Nの行列を$I$とすると、<br>
リンクをたどったアクセスが $0.85*B$<br>ランダムなアクセスが $0.15*I$<br>
となるので、スコアを分け与える割合の行列Cは、以下のように表せる。

    $C=0.85*B + 0.15*I$

In [2]:
import numpy as np

p = 0.85
N = len(id_to_title)
A = np.zeros((N,N))
B = np.zeros((N,N))
C = np.zeros((N,N))
I = np.ones((N,N))/N

#1
for Id, lst in id_to_nextIDs.items():
    for nextId in lst:
        A[id_to_index[Id]][id_to_index[nextId]] = 1
        
        
#2, 3
for i in range(N):
    if sum(A[i]) == 0:
        B[i][:] = 1/N
    else:
        B[i] = A[i]/sum(A[i])
        

#4
C = p*B + (1-p)*I

### 3. 各頂点のスコアを求める

各頂点のスコアの行列(1×N)と、上で求めた行列(N×N)をかけることで、分配終了後のスコアの行列(1×N)を求める。

In [3]:
#元のスコアは全て1
score = np.ones((N))

#30ラウンド回した後のスコアが高い順に出力
for i in range(30):
    score = score@C
    
lst = [[id_to_title[i], score[id_to_index[i]]] for i in id_to_title.keys()]
for pair in sorted(lst, key=lambda x: x[1], reverse=True):
    print(f"{pair[0]} : {pair[1]}")

国際宇宙ステーション : 3.182113724306706
歩道 : 2.6672523285488507
道路交通法 : 2.6672523285488507
環境 : 2.326552476337514
コンピュータ : 2.0335826541913935
人工知能 : 1.574944965522764
Google : 1.5161374187676924
ロボット : 1.3745001564855908
量子コンピュータ : 1.2349757126136278
Android : 1.160448362741174
ゲーム : 1.0766798452690842
昆虫 : 0.7402493261182769
じゃんけん : 0.7069951839478356
健康 : 0.6773989923420302
パワードスーツ : 0.6136692226630455
カレー : 0.5758299349338931
セグウェイ : 0.5260682210289404
学習 : 0.5025207207675894
情報 : 0.5025207207675894
ロボット工学三原則 : 0.4781744329341551
深海 : 0.4781744329341551
カメラ : 0.4756704471098385
アイスクリーム : 0.4756704471098385
キットカット : 0.4756704471098385
タイムマシン : 0.40069221727244875
和歌 : 0.31125105592545466
ツキノワグマ : 0.31125105592545466
東京 : 0.31125105592545466
ラーメン : 0.3112510559254546
おにぎり : 0.3112510559254546
